## roop换脸
此文件的作用是使用roop插件进行批量换脸，只需要定义源文件夹，目标文件夹，脸部参考图片，即可进行批量换脸，换脸后的图片会保存在目标文件夹中。

In [ ]:
import json
import random
import uuid
import urllib.request
import urllib.parse
from urllib.error import URLError
import io
import os
import time
from PIL.PngImagePlugin import PngInfo
from PIL import Image
import hashlib


## import json file

In [ ]:
promptJson = 'api.json'

with open(promptJson) as f:
    prompt = json.load(f)

print(str(prompt))

In [ ]:
server_address = "127.0.0.1:8188"
client_id = str(uuid.uuid4())


In [ ]:
def changeImageSize(width, height):
    # 最小的宽高为 896
    limited = 896
    minSize = min(width, height)

    # 缩放宽高，使得最窄的边等于limited
    if minSize > limited:
        scale = minSize / limited
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = limited / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数
    widthScale = int(widthScale/64) * 64
    heightScale = int(heightScale/64) * 64

    return widthScale, heightScale

def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False


def queue_prompt(prompt):
    while True:
        try:
            p = {"prompt": prompt, "client_id": client_id}
            data = json.dumps(p).encode('utf-8')
            req = urllib.request.Request("http://{}/prompt".format(server_address), data=data)
            return json.loads(urllib.request.urlopen(req, timeout=5).read())
        except URLError:
            print("连接超时，正在重试...")

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def handle_whitespace(string: str):
    return string.strip().replace("\n", " ").replace("\r", " ").replace("\t", " ")

def parse_name(ckpt_name):
    path = ckpt_name
    filename = path.split("/")[-1]
    filename = filename.split(".")[:-1]
    filename = ".".join(filename)
    return filename


def calculate_sha256(file_path):
    sha256_hash = hashlib.sha256()

    with open(file_path, "rb") as f:
        # Read the file in chunks to avoid loading the entire file into memory
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

    return sha256_hash.hexdigest()

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
                output_images[node_id] = images_output

    return output_images
    
def saveImages(result, imageDir,imageName,comment,previewImage=0): 
    metadata = PngInfo()
    metadata.add_text("parameters", comment)  
    k = 0 
    for node_id in result:    
        if k < previewImage:
            k += 1
            continue    
        for image_data in result[node_id]:            
            image = Image.open(io.BytesIO(image_data))
            imagePath = imageDir + "/" + imageName + f"_{k}" + ".png"
            image.save(imagePath,pnginfo=metadata) 
            k += 1
    

In [ ]:
# 子文件夹
subDir = "BoA"

# 原图文件夹
originalDir = f"E:/roop-face/source-video/temp/test-video"
# 设置图片保存路径
imageDir = f"E:/roop-face/source-video/temp/test-video-output"
# 如果文件夹不存在，创建文件夹
if not os.path.exists(imageDir):
    os.mkdir(imageDir)
    
# 这里是要替换的人脸图片
faceReferenceImagePath = "c:/Users/BigHippo78/Documents/WeChat Files/highmore78/FileStorage/File/2023-10/奶茶多多/奶茶多多/face.png"

In [ ]:

import websocket

ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))

# 这些参数是乱写的
positive = ''
negative = ''
steps = 35
sampler_name = 'dpmpp_2m_sde'
scheduler = 'karras'
cfg = 7
seed = random.randint(0, 1000000)

count = 0

batch_size = 1
# 遍历原图文件夹里所有的图片
for originalImag in os.listdir(originalDir):
        # 如果不是jpg或png文件，跳过
        if not (originalImag.endswith(".jpg") or originalImag.endswith(".png") or originalImag.endswith(".jpeg")):
            continue
        
        originalImagePath = originalDir + "/" + originalImag
        # 如果文件名包含_H或者_B，跳过
        if originalImag.find("_H") != -1 or originalImag.find("_B") != -1:
            print(f"{originalImag}包含_H或者_B,判定为mask文件，跳过")
        
        # 读取原图片名字，去掉后缀.png
        originalImagName = originalImag.split(".")[0]
        # 检测目标文件夹里是否已经存在该图片，如果存在，跳过
        if checkImageExsit(imageDir, originalImagName):
            print(f"{originalImagName}已经存在，跳过")
            continue 
        
        # roop只需要两个参数，就是原图片和脸部参考图片
        prompt['35']['inputs']['image'] = faceReferenceImagePath
        prompt['12']['inputs']['image'] = originalImagePath       
        
        comment = ''    
        
        # 连接api，生成并保存图片
        start_time = time.time()        
        images = get_images(ws, prompt)   
        saveImages(images, imageDir, originalImagName, comment=comment,previewImage=0)     
        end_time = time.time()
        print(f"耗时{end_time-start_time}秒")
        count += 1
        print(f"已经生成{count}张图片")
        